🌈**in this notebook I am trying to visualize the Entire Publication Text Corpus and observe relation between datasets and Text**.🌈

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
from tqdm.autonotebook import tqdm
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
def append_text(filename, train_files_path =train_files_path, output = 'text'):
    json_path = os.path.join(train_files_path, (filename+ '.json'))
    Heading = []
    Content = []
    Combined = []
    with open (json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            Heading.append(data.get('section_title'))
            Content.append(data.get('text'))
            Combined.append(data.get('section_title'))
            Combined.append(data.get('text'))
            
    all_headings = ' '.join(Heading)
    all_contents = ' '.join(Content)
    all_data = ' '.join(Combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data        

In [ ]:
tqdm.pandas()
train_df['text'] = train_df['Id'].progress_apply(append_text)

In [ ]:
train_df.info()

In [ ]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
%matplotlib inline
def cleantext(df): 
    
    train_df['cleaned_text'] = train_df['text'].replace(r'\'|\"|\,|\.|\?|\+|\-|\/|\=|\(|\)|\n|"', '', regex=True)
    train_df['cleaned_text'] = train_df['cleaned_text'].replace("  ", " ")
    
    # convert tweets to lowercase
    train_df['cleaned_text'] = train_df['cleaned_text'].str.lower()
    
     #remove_symbols
    train_df['cleaned_text']  = train_df['cleaned_text'].replace(r'[^a-zA-Z0-9]', " ", regex=True)
    
    #remove punctuations 
    train_df['cleaned_text'] = train_df['cleaned_text'].replace(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+',"", regex = True)
    
    #remove_URL(x):
    train_df['cleaned_text']  = train_df['cleaned_text'].replace(r'https.*$', "", regex = True)
    
    #remove stopwords and words_to_remove
    
    mystopwords = set(stopwords.words('english'))
    
    train_df['fully_cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in mystopwords]))
    

    return df

train_df = cleantext(train_df)

In [ ]:
from collections import defaultdict

words_in_text_by_dataset = defaultdict(list)

for _, row in train_df.iterrows():
    words_in_text_by_dataset[row['dataset_title']].extend(row['fully_cleaned_text'].split())

# Defining our word cloud drawing function
def wordcloud_draw(data, color = 'white'):
    wordcloud = WordCloud(stopwords = STOPWORDS,
                          background_color = color,
                          width = 3000,
                          height = 2000
                         ).generate(' '.join(data))
    plt.figure(1, figsize = (12, 8))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

for dataset_title in train_df['dataset_title'].unique():
    print("WordCloud for Publications Text mentioning", dataset_title, ":")
    wordcloud_draw(words_in_text_by_dataset[dataset_title])